In [1]:
import pandas as pd
import numpy as np
import glob
import os
from openpyxl import load_workbook
pd.options.display.float_format = '{:,.2f}'.format

In [2]:
start_date = '2023-05-01'
final_date = '2023-05-31'
query_error_year = 2023
query_error_month = 5

# INFORMACION DE  CUBETA ERROR

In [3]:
query_error = pd.read_csv(r'C:\Users\Cristian.Aparicio\Documents\POLIZAS\2023\05. MAYO\DATA\ERROR MAYO 2023.csv',
                          dtype={'report_id':object})
query_error.rename(columns ={'MAX(report_date_local)':'report_date_local'}, inplace=True)
query_error['report_date_local']= pd.to_datetime(query_error['report_date_local'])
query_error['ultimo_mov'] = pd.to_datetime(query_error['ultimo_mov'])
query_error['merchant_id_query'] = query_error['merchant_id']
query_error['ultimo_mov_date_only']= pd.to_datetime(query_error['ultimo_mov']).dt.date
query_error['day'] = query_error['ultimo_mov'].dt.day
query_error['month'] = query_error['ultimo_mov'].dt.month
query_error['year'] = query_error['ultimo_mov'].dt.year
# query_error.loc[((query_error['day'] == 30 )
#                 & (query_error['month'] == 7)
#                 & (query_error['year'] == 2022)), "mes_cierre"] = 'Actual'
# query_error.loc[(query_error['day'] == 31 )
#             & (query_error['month'] == 7 )
#             & (query_error['year'] == 2022), "mes_cierre"] = 'Actual'
query_error.loc[(query_error['month'] == query_error_month )
            & (query_error['year'] == query_error_year), "mes_cierre"] = 'Actual'
query_error.head()

,Unnamed: 0,merchant_id,merchant_name,report_id,report_date_local,amount,status,error_description,ultimo_mov,month,year,status_cubeta,esquema_query,merchant_id_query,ultimo_mov_date_only,day,mes_cierre
0,0,b088c4b4-29dd-42c8-bd29-1eda61f73973,Mscelanea Dulce Maria,GTS92WH86,2015-11-05,734.34,ERROR,swap devuelto,2017-08-15,8,2017,ERROR,ERROR_USUAL,b088c4b4-29dd-42c8-bd29-1eda61f73973,2017-08-15,15,NaN
1,1,b088c4b4-29dd-42c8-bd29-1eda61f73973,Mscelanea Dulce Maria,8VQRDV39G,2015-11-06,597.26,ERROR,swap devuelto,2017-08-15,8,2017,ERROR,ERROR_USUAL,b088c4b4-29dd-42c8-bd29-1eda61f73973,2017-08-15,15,NaN
2,2,d1087b80-5383-4b90-8738-bf309a1ce297,Grupo Solutec,82G5CYSR7,2015-12-21,"9,160.77",ERROR,failed swap,2017-11-01,11,2017,ERROR,ERROR_USUAL,d1087b80-5383-4b90-8738-bf309a1ce297,2017-11-01,1,NaN
3,3,8f666f7d-5eb8-4367-9c52-5564d6aca0c5,Ediciones Malinalco S de RL de CV,XWRKLC2ZL,2016-02-08,"1,216.98",ERROR,confirmar la cuenta,2017-05-12,5,2017,ERROR,ERROR_USUAL,8f666f7d-5eb8-4367-9c52-5564d6aca0c5,2017-05-12,12,NaN
4,4,8f666f7d-5eb8-4367-9c52-5564d6aca0c5,Ediciones Malinalco S de RL de CV,D9468C7W4,2016-02-16,"1,993.14",ERROR,confirmar la cuenta,2017-05-12,5,2017,ERROR,ERROR_USUAL,8f666f7d-5eb8-4367-9c52-5564d6aca0c5,2017-05-12,12,NaN


In [4]:
query_error.shape

(133472, 17)

In [5]:
query_error.dtypes

Unnamed: 0                       int64
merchant_id                     object
merchant_name                   object
report_id                       object
report_date_local       datetime64[ns]
amount                         float64
status                          object
error_description               object
ultimo_mov              datetime64[ns]
month                            int64
year                             int64
status_cubeta                   object
esquema_query                   object
merchant_id_query               object
ultimo_mov_date_only            object
day                              int64
mes_cierre                      object
dtype: object

In [6]:
query_error['esquema_query'].unique()

array(['ERROR_USUAL', 'SIN_CLABE'], dtype=object)

In [7]:
query_error['status_cubeta'].unique()

array(['ERROR', 'NEW'], dtype=object)

# FILTRAR INFORMACION DE ACUERDO AL MES EN CURSO 

In [8]:
query_error_current_month = query_error[(query_error['mes_cierre'] == 'Actual')]
query_error_current_month.sort_values(by=['day'],inplace=True)
query_error_current_month.head()

C:\Users\Cristian.Aparicio\AppData\Local\Temp\ipykernel_14008\3502651561.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  query_error_current_month.sort_values(by=['day'],inplace=True)


,Unnamed: 0,merchant_id,merchant_name,report_id,report_date_local,amount,status,error_description,ultimo_mov,month,year,status_cubeta,esquema_query,merchant_id_query,ultimo_mov_date_only,day,mes_cierre
117721,117721,962a6269-3b7b-453a-a973-2b746b0b4909,NaN,APPFQTDKZ,2023-04-29,443.67,ERROR,DEV.SPEIEXCEDE EL LNMITE DE AB,2023-05-01,5,2023,NEW,ERROR_USUAL,962a6269-3b7b-453a-a973-2b746b0b4909,2023-05-01,1,Actual
118372,118372,af5a71ac-8639-420d-b513-0395c0c4f66d,NaN,ATFHQVKWX,2023-05-01,603.69,ERROR,Bank account information not available,2023-05-01,5,2023,NEW,SIN_CLABE,af5a71ac-8639-420d-b513-0395c0c4f66d,2023-05-01,1,Actual
118373,118373,1493da6d-4ee7-4b86-a9d6-56b24243f66b,NaN,A62D642LC,2023-05-01,129.36,ERROR,Bank account information not available,2023-05-01,5,2023,NEW,SIN_CLABE,1493da6d-4ee7-4b86-a9d6-56b24243f66b,2023-05-01,1,Actual
118374,118374,e9513730-acbe-4fa3-964a-f09df2b80c6e,NaN,AGBCM9MZ9,2023-05-01,"1,812.03",ERROR,Bank account information not available,2023-05-01,5,2023,NEW,SIN_CLABE,e9513730-acbe-4fa3-964a-f09df2b80c6e,2023-05-01,1,Actual
118375,118375,6383fe37-ae8d-47ce-bda5-dc06bf1c176f,NaN,AQ7LGNJD3,2023-05-01,"1,302.25",ERROR,Bank account information not available,2023-05-01,5,2023,NEW,SIN_CLABE,6383fe37-ae8d-47ce-bda5-dc06bf1c176f,2023-05-01,1,Actual


In [9]:
query_error_current_month.shape

(15627, 17)

In [10]:
query_error_current_month['month'].unique()

array([5], dtype=int64)

In [11]:
query_error_current_month.columns

Index(['Unnamed: 0', 'merchant_id', 'merchant_name', 'report_id',
       'report_date_local', 'amount', 'status', 'error_description',
       'ultimo_mov', 'month', 'year', 'status_cubeta', 'esquema_query',
       'merchant_id_query', 'ultimo_mov_date_only', 'day', 'mes_cierre'],
      dtype='object')

In [12]:
duplicates_query_error = query_error_current_month[query_error_current_month.duplicated(['report_id'] , keep=False)]
duplicates_query_error.head()

,Unnamed: 0,merchant_id,merchant_name,report_id,report_date_local,amount,status,error_description,ultimo_mov,month,year,status_cubeta,esquema_query,merchant_id_query,ultimo_mov_date_only,day,mes_cierre


In [13]:
duplicates_query_error.shape

(0, 17)

# PAGOS NEW DE MESES ANTERIORES

In [14]:
query_new_last_month = query_error[(query_error['mes_cierre'] != 'Actual') & (query_error['status_cubeta'] == 'NEW')]
query_new_last_month.sort_values(by=['day'],inplace=True)
query_new_last_month.head()

C:\Users\Cristian.Aparicio\AppData\Local\Temp\ipykernel_14008\3381233170.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  query_new_last_month.sort_values(by=['day'],inplace=True)


,Unnamed: 0,merchant_id,merchant_name,report_id,report_date_local,amount,status,error_description,ultimo_mov,month,year,status_cubeta,esquema_query,merchant_id_query,ultimo_mov_date_only,day,mes_cierre


In [15]:
query_new_last_month.shape

(0, 17)

# INFORMACIÓN QUERIES - PAGOS

In [16]:
name_header = ['merchant_id','merchant_name','user_email','report_id','report_date_local','amount','status','bank_name'
               ,'payment_date_local','bank_account_owner_name','bank_account_number','swap']

### NEXT MONTH

In [17]:
# TRADICIONAL
# trad_files_csv_2021_may = glob.glob(r'G:\My Drive\Settlement\5.-HISTORICO FILES PAGOS\2021\06. JUNE\QUERIES\*.csv', recursive=False)
# trad_files_txt_2021_apr = glob.glob(r'G:\My Drive\Settlement\5.-HISTORICO FILES PAGOS\2021\04. APRIL\QUERIES\*.txt', recursive=False)
# FAST PAYMENTS
# fp_files_csv_2021_may = glob.glob(r'C:\Users\Cristian.Aparicio\Documents\FAST PAYMENTS\2021\06. JUNE\QUERIES\*.csv', recursive=False)

In [18]:
# disp_files_csv_2021_may = pd.concat([pd.read_csv(f,dtype={'bank_account_number':object,'report_id':object})
#                             .assign(file_name=os.path.basename(f)) for f in trad_files_csv_2021_may])
# disp_files_csv_2021_may['date_query'] = disp_files_csv_2021_may["file_name"].str[-14:-4]
# disp_files_csv_2021_may['date_query']= pd.to_datetime(disp_files_csv_2021_may['date_query'])

In [19]:
# fp_disp_files_csv_2021_may = pd.concat([pd.read_csv(f,dtype={'bank_account_number':object,'report_id':object})
#                             .assign(file_name=os.path.basename(f)) for f in fp_files_csv_2021_may])
# fp_disp_files_csv_2021_may['date_query'] = fp_disp_files_csv_2021_may["file_name"].str[-14:-4]
# fp_disp_files_csv_2021_may['date_query']= pd.to_datetime(fp_disp_files_csv_2021_may['date_query']) 

### CURRENT MONTH

In [20]:
# TRADICIONAL
trad_files_csv_2022_current = glob.glob(r'G:\.shortcut-targets-by-id\1Oe28Mf9uSidMamjllDen2gkNnC59eSUr\3.-Settlement\5.-HISTORICO FILES PAGOS\01. TRADICIONAL\2023\05. MAY\QUERIES\*.csv', recursive=False)
# trad_files_txt_2021_apr = glob.glob(r'G:\My Drive\Settlement\5.-HISTORICO FILES PAGOS\2021\04. APRIL\QUERIES\*.txt', recursive=False)
# FAST PAYMENTS
fp_files_csv_2022_current = glob.glob(r'G:\.shortcut-targets-by-id\1Oe28Mf9uSidMamjllDen2gkNnC59eSUr\3.-Settlement\5.-HISTORICO FILES PAGOS\02. FAST PAYMENTS\2023\05. MAYO\QUERIES\*.csv', recursive=False)

In [21]:
disp_files_csv_2022_current = pd.concat([pd.read_csv(f,dtype={'bank_account_number':object,'report_id':object})
                            .assign(file_name=os.path.basename(f)) for f in trad_files_csv_2022_current])
disp_files_csv_2022_current['date_query'] = disp_files_csv_2022_current["file_name"].str[-14:-4]
disp_files_csv_2022_current['date_query']= pd.to_datetime(disp_files_csv_2022_current['date_query'])

C:\Users\Cristian.Aparicio\AppData\Local\Temp\ipykernel_14008\1256843064.py:1: DtypeWarning: Columns (0,2,3,4) have mixed types. Specify dtype option on import or set low_memory=False.
  disp_files_csv_2022_current = pd.concat([pd.read_csv(f,dtype={'bank_account_number':object,'report_id':object})
C:\Users\Cristian.Aparicio\AppData\Local\Temp\ipykernel_14008\1256843064.py:1: DtypeWarning: Columns (0,2,3,4) have mixed types. Specify dtype option on import or set low_memory=False.
  disp_files_csv_2022_current = pd.concat([pd.read_csv(f,dtype={'bank_account_number':object,'report_id':object})


In [22]:
fp_disp_files_csv_2022_current = pd.concat([pd.read_csv(f,dtype={'bank_account_number':object,'report_id':object})
                            .assign(file_name=os.path.basename(f)) for f in fp_files_csv_2022_current])
fp_disp_files_csv_2022_current['date_query'] = fp_disp_files_csv_2022_current["file_name"].str[-14:-4]
fp_disp_files_csv_2022_current['date_query']= pd.to_datetime(fp_disp_files_csv_2022_current['date_query']) 

### LAST MONTH

In [23]:
# TRADICIONAL
trad_files_csv_2021_last = glob.glob(r'G:\.shortcut-targets-by-id\1Oe28Mf9uSidMamjllDen2gkNnC59eSUr\3.-Settlement\5.-HISTORICO FILES PAGOS\01. TRADICIONAL\2023\04. APRIL\QUERIES\*.csv', recursive=False)
# trad_files_txt_2021_apr = glob.glob(r'G:\My Drive\Settlement\5.-HISTORICO FILES PAGOS\2021\04. APRIL\QUERIES\*.txt', recursive=False)
# FAST PAYMENTS
fp_files_csv_2021_last = glob.glob(r'G:\.shortcut-targets-by-id\1Oe28Mf9uSidMamjllDen2gkNnC59eSUr\3.-Settlement\5.-HISTORICO FILES PAGOS\02. FAST PAYMENTS\2023\04. ABRIL\QUERIES\*.csv', recursive=False)

In [24]:
disp_files_csv_2021_last = pd.concat([pd.read_csv(f,dtype={'bank_account_number':object,'report_id':object})
                            .assign(file_name=os.path.basename(f)) for f in trad_files_csv_2021_last])
disp_files_csv_2021_last['date_query'] = disp_files_csv_2021_last["file_name"].str[-14:-4]
disp_files_csv_2021_last['date_query']= pd.to_datetime(disp_files_csv_2021_last['date_query'])

In [25]:
fp_disp_files_csv_2021_last = pd.concat([pd.read_csv(f,dtype={'bank_account_number':object,'report_id':object})
                            .assign(file_name=os.path.basename(f)) for f in fp_files_csv_2021_last])
fp_disp_files_csv_2021_last['date_query'] = fp_disp_files_csv_2021_last["file_name"].str[-14:-4]
fp_disp_files_csv_2021_last['date_query']= pd.to_datetime(fp_disp_files_csv_2021_last['date_query']) 

### SPECIAL MONTH

In [26]:
# TRADICIONAL
trad_files_csv_2021_spec = glob.glob(r'G:\.shortcut-targets-by-id\1Oe28Mf9uSidMamjllDen2gkNnC59eSUr\3.-Settlement\5.-HISTORICO FILES PAGOS\01. TRADICIONAL\2023\03. MARCH\QUERIES\*.csv', recursive=False)
# trad_files_txt_2021_apr = glob.glob(r'G:\My Drive\Settlement\5.-HISTORICO FILES PAGOS\2021\04. APRIL\QUERIES\*.txt', recursive=False)
# FAST PAYMENTS
fp_files_csv_2021_spec = glob.glob(r'G:\.shortcut-targets-by-id\1Oe28Mf9uSidMamjllDen2gkNnC59eSUr\3.-Settlement\5.-HISTORICO FILES PAGOS\02. FAST PAYMENTS\2023\03. MARZO\QUERIES\*.csv', recursive=False)

In [27]:
disp_files_csv_2021_spec = pd.concat([pd.read_csv(f,dtype={'bank_account_number':object,'report_id':object})
                            .assign(file_name=os.path.basename(f)) for f in trad_files_csv_2021_spec])
disp_files_csv_2021_spec['date_query'] = disp_files_csv_2021_spec["file_name"].str[-14:-4]
disp_files_csv_2021_spec['date_query']= pd.to_datetime(disp_files_csv_2021_spec['date_query'])

In [28]:
fp_disp_files_csv_2021_spec = pd.concat([pd.read_csv(f,dtype={'bank_account_number':object,'report_id':object})
                            .assign(file_name=os.path.basename(f)) for f in fp_files_csv_2021_spec])
fp_disp_files_csv_2021_spec['date_query'] = fp_disp_files_csv_2021_spec["file_name"].str[-14:-4]
fp_disp_files_csv_2021_spec['date_query']= pd.to_datetime(fp_disp_files_csv_2021_spec['date_query']) 

## ALL FILES

In [29]:
allfiles = pd.concat([disp_files_csv_2022_current,fp_disp_files_csv_2022_current
                     ,disp_files_csv_2021_last,fp_disp_files_csv_2021_last,
                     disp_files_csv_2021_spec,fp_disp_files_csv_2021_spec])
allfiles.reset_index()
allfiles.fillna('BLANK',inplace=True)
allfiles['date_query']= pd.to_datetime(allfiles['date_query'])
allfiles['report_date_local']= pd.to_datetime(allfiles['report_date_local'])
allfiles.loc[(allfiles['file_name'].str.contains('FP'),'tipo_dispersion')] = 'FP'
allfiles.loc[(allfiles['file_name'].str.contains('SWAP'),'tipo_dispersion')] = 'TRAD'
allfiles.loc[(allfiles['file_name'].str.contains('BNTE'),'tipo_dispersion')] = 'TRAD'
allfiles.loc[(allfiles['file_name'].str.contains('SANT'),'tipo_dispersion')] = 'TRAD'
allfiles.sort_values(by=['date_query','report_id'],inplace=True)
allfiles.drop(['status'],axis=1,inplace=True)
allfiles.head()

,type,merchant_id,merchant_token,merchant_name,user_email,report_id,report_date_local,amount,bank_name,payment_date_local,bank_account_owner_name,bank_account_number,batch_created_date,disbursement_method,swap,file_name,date_query,tipo_dispersion
39871,DAILY,652400c6-31ac-479d-a667-0db7225b1c0a,80af32bb-cd90-4d11-a3da-126ea9822fcb,Abarrotes Flor,iro_68@hotmail.com,22237PFT4,2023-03-01,279.51,BANAMEX,2023-03-01 11:49:47,Ignacio Rodrigues,'002905701065656419,2023-03-01 10:07:37,SANTANDER,"Ignacio Rodrigues ,,279.51,002905701065656419,...",SANT-1 2023-03-01.csv,2023-03-01,TRAD
51506,DAILY,82ca1460-c459-41a1-ba50-f412dfd2d12a,e899a771-2913-4e6e-bd1e-89598302a789,TheBarbersLM,lorenamendez21461@hotmail.com,222D4MHCS,2023-03-01,383.30,BBVA Bancomer,2023-03-01 11:46:47,Doris Esther Guerra Blanco,'012180015066699772,2023-03-01 10:09:03,SANTANDER,"Doris Esther Guerra Blanco,,383.30,01218001506...",SANT-1 2023-03-01.csv,2023-03-01,TRAD
24440,DAILY,3e3d2b7d-eed6-4dd5-b251-0aa310f0c85f,a6a3d955-a08c-4b6d-bccb-71d498ed2528,Dulces andy,jrgk7232@gmail.com,222KVTC6P,2023-03-01,"1,437.36",Banco Azteca,2023-03-01 11:43:46,Jorge Luis Rada Malaga,'127028013605334641,2023-03-01 10:05:53,SANTANDER,"Jorge Luis Rada Malaga,,1437.36,12702801360533...",SANT-1 2023-03-01.csv,2023-03-01,TRAD
1192,DAILY,031c670b-6626-40a1-89ea-fc5ed1bbec8d,99d051b9-bceb-46a4-a82f-21cac8c9530b,Eridental,draerival@hotmail.com,222L2BHB2,2023-03-01,"2,491.42",SANTANDER,2023-03-01 11:42:10,Erika Yaneth Valdez Muñoz,'014580606037523321,2023-03-01 10:02:57,SANTANDER,"Erika Yaneth Valdez Muñoz,,2491.42,01458060603...",SANT-1 2023-03-01.csv,2023-03-01,TRAD
42433,DAILY,6bce3214-9af4-4c92-a5e0-dfc7f749e55e,2894580e-15a6-4fbe-ae0b-3cf25b402ec1,Abarrotes salas,m.josegabriel38@gmail.com,222N6GY5F,2023-03-01,35.46,BBVA Bancomer,2023-03-01 11:42:58,Maria Margarita Guzmn Salas,'012180015522923814,2023-03-01 10:07:59,SANTANDER,"Maria Margarita Guzmn Salas,,35.46,01218001552...",SANT-1 2023-03-01.csv,2023-03-01,TRAD


In [30]:
def tipo_pago(allfiles):
    if allfiles['date_query'] == allfiles['report_date_local']:
        return 'Pago del día'
    else:
        return 'Reprogramación'
allfiles['tipo_pago'] = allfiles.apply(tipo_pago, axis=1)

In [31]:
allfiles.head()

,type,merchant_id,merchant_token,merchant_name,user_email,report_id,report_date_local,amount,bank_name,payment_date_local,bank_account_owner_name,bank_account_number,batch_created_date,disbursement_method,swap,file_name,date_query,tipo_dispersion,tipo_pago
39871,DAILY,652400c6-31ac-479d-a667-0db7225b1c0a,80af32bb-cd90-4d11-a3da-126ea9822fcb,Abarrotes Flor,iro_68@hotmail.com,22237PFT4,2023-03-01,279.51,BANAMEX,2023-03-01 11:49:47,Ignacio Rodrigues,'002905701065656419,2023-03-01 10:07:37,SANTANDER,"Ignacio Rodrigues ,,279.51,002905701065656419,...",SANT-1 2023-03-01.csv,2023-03-01,TRAD,Pago del día
51506,DAILY,82ca1460-c459-41a1-ba50-f412dfd2d12a,e899a771-2913-4e6e-bd1e-89598302a789,TheBarbersLM,lorenamendez21461@hotmail.com,222D4MHCS,2023-03-01,383.30,BBVA Bancomer,2023-03-01 11:46:47,Doris Esther Guerra Blanco,'012180015066699772,2023-03-01 10:09:03,SANTANDER,"Doris Esther Guerra Blanco,,383.30,01218001506...",SANT-1 2023-03-01.csv,2023-03-01,TRAD,Pago del día
24440,DAILY,3e3d2b7d-eed6-4dd5-b251-0aa310f0c85f,a6a3d955-a08c-4b6d-bccb-71d498ed2528,Dulces andy,jrgk7232@gmail.com,222KVTC6P,2023-03-01,"1,437.36",Banco Azteca,2023-03-01 11:43:46,Jorge Luis Rada Malaga,'127028013605334641,2023-03-01 10:05:53,SANTANDER,"Jorge Luis Rada Malaga,,1437.36,12702801360533...",SANT-1 2023-03-01.csv,2023-03-01,TRAD,Pago del día
1192,DAILY,031c670b-6626-40a1-89ea-fc5ed1bbec8d,99d051b9-bceb-46a4-a82f-21cac8c9530b,Eridental,draerival@hotmail.com,222L2BHB2,2023-03-01,"2,491.42",SANTANDER,2023-03-01 11:42:10,Erika Yaneth Valdez Muñoz,'014580606037523321,2023-03-01 10:02:57,SANTANDER,"Erika Yaneth Valdez Muñoz,,2491.42,01458060603...",SANT-1 2023-03-01.csv,2023-03-01,TRAD,Pago del día
42433,DAILY,6bce3214-9af4-4c92-a5e0-dfc7f749e55e,2894580e-15a6-4fbe-ae0b-3cf25b402ec1,Abarrotes salas,m.josegabriel38@gmail.com,222N6GY5F,2023-03-01,35.46,BBVA Bancomer,2023-03-01 11:42:58,Maria Margarita Guzmn Salas,'012180015522923814,2023-03-01 10:07:59,SANTANDER,"Maria Margarita Guzmn Salas,,35.46,01218001552...",SANT-1 2023-03-01.csv,2023-03-01,TRAD,Pago del día


In [32]:
allfiles.shape

(9352472, 19)

# CRUCE 1 - MES ACTUAL

In [33]:
cruce_1 = allfiles.merge(query_error_current_month[['report_id','amount','ultimo_mov','ultimo_mov_date_only','month','status'
                                                   ,'error_description','esquema_query','status_cubeta']],on='report_id',how='left')
cruce_1['month'].fillna('Not found',inplace=True)
cruce_1.head()

,type,merchant_id,merchant_token,merchant_name,user_email,report_id,report_date_local,amount_x,bank_name,payment_date_local,...,tipo_dispersion,tipo_pago,amount_y,ultimo_mov,ultimo_mov_date_only,month,status,error_description,esquema_query,status_cubeta
0,DAILY,652400c6-31ac-479d-a667-0db7225b1c0a,80af32bb-cd90-4d11-a3da-126ea9822fcb,Abarrotes Flor,iro_68@hotmail.com,22237PFT4,2023-03-01,279.51,BANAMEX,2023-03-01 11:49:47,...,TRAD,Pago del día,NaN,NaT,NaN,Not found,NaN,NaN,NaN,NaN
1,DAILY,82ca1460-c459-41a1-ba50-f412dfd2d12a,e899a771-2913-4e6e-bd1e-89598302a789,TheBarbersLM,lorenamendez21461@hotmail.com,222D4MHCS,2023-03-01,383.30,BBVA Bancomer,2023-03-01 11:46:47,...,TRAD,Pago del día,NaN,NaT,NaN,Not found,NaN,NaN,NaN,NaN
2,DAILY,3e3d2b7d-eed6-4dd5-b251-0aa310f0c85f,a6a3d955-a08c-4b6d-bccb-71d498ed2528,Dulces andy,jrgk7232@gmail.com,222KVTC6P,2023-03-01,"1,437.36",Banco Azteca,2023-03-01 11:43:46,...,TRAD,Pago del día,NaN,NaT,NaN,Not found,NaN,NaN,NaN,NaN
3,DAILY,031c670b-6626-40a1-89ea-fc5ed1bbec8d,99d051b9-bceb-46a4-a82f-21cac8c9530b,Eridental,draerival@hotmail.com,222L2BHB2,2023-03-01,"2,491.42",SANTANDER,2023-03-01 11:42:10,...,TRAD,Pago del día,NaN,NaT,NaN,Not found,NaN,NaN,NaN,NaN
4,DAILY,6bce3214-9af4-4c92-a5e0-dfc7f749e55e,2894580e-15a6-4fbe-ae0b-3cf25b402ec1,Abarrotes salas,m.josegabriel38@gmail.com,222N6GY5F,2023-03-01,35.46,BBVA Bancomer,2023-03-01 11:42:58,...,TRAD,Pago del día,NaN,NaT,NaN,Not found,NaN,NaN,NaN,NaN


In [34]:
cruce_1.shape

(9352472, 27)

In [35]:
duplicates_cruce_1_previo = cruce_1[cruce_1.duplicated(['report_id'] , keep=False)]
duplicates_cruce_1_previo.head()

,type,merchant_id,merchant_token,merchant_name,user_email,report_id,report_date_local,amount_x,bank_name,payment_date_local,...,tipo_dispersion,tipo_pago,amount_y,ultimo_mov,ultimo_mov_date_only,month,status,error_description,esquema_query,status_cubeta
312,INSTANT,6ce64926-c71d-4998-b464-962112a46ad1,ebddf3fc-c7e9-47c0-a434-30031a831e9c,Gauchos Grill,abigail_conde@hotmail.com,24YNSMG4M,2023-03-01,104.45,"Opciones Empresariales del Noreste, S.A. de C....",2023-03-01 14:12:37,...,FP,Pago del día,NaN,NaT,NaN,Not found,NaN,NaN,NaN,NaN
323,DAILY,90d3c380-baca-47cb-a0b8-ef074b320b87,917d2f5e-8c64-4be5-aeed-19ae4fac1e2d,Miscelanea Katy,omarg5870@gmail.com,252H3FMST,2023-03-01,65.16,HSBC,2023-03-01 23:30:46,...,TRAD,Pago del día,NaN,NaT,NaN,Not found,NaN,NaN,NaN,NaN
1147,DAILY,734e4fab-3874-41b2-b722-a70b5679b4ac,be88d98a-cd97-4f91-b5e6-172d12c68097,El Paraiso,joaquinguerrero696@gmail.com,2DQR2F7XJ,2023-03-01,474.34,"BBVA Bancomer, S.A.",2023-03-01 11:44:18,...,TRAD,Pago del día,NaN,NaT,NaN,Not found,NaN,NaN,NaN,NaN
1749,DAILY,2beba981-35dd-44ef-a568-688bb24025d0,853d453c-6918-49ee-bf98-309527207a36,Marush,0215rubi@gmail.com,2L4X2236V,2023-03-01,1.20,"Banco Santander, S.A.",2023-03-01 11:48:34,...,TRAD,Pago del día,NaN,NaT,NaN,Not found,NaN,NaN,NaN,NaN
2353,DAILY,d2d62c7d-4bdb-4a83-ab2a-98d5a03c7d60,ff72b8de-7445-45ea-bafc-5d0770f607cc,Tiendita Lolita,lolita.marquez.alva@gmail.com,2RVDPYMDL,2023-02-14,819.55,BANORTE,2023-03-01 13:06:35,...,TRAD,Reprogramación,NaN,NaT,NaN,Not found,NaN,NaN,NaN,NaN


In [36]:
duplicates_cruce_1_previo.shape

(59317, 27)

In [37]:
cruce_1_final = cruce_1[(cruce_1['month'] != 'Not found')]
cruce_1_final.drop_duplicates(subset='report_id',keep='last', inplace=True) 
cruce_1_final.head()

C:\Users\Cristian.Aparicio\AppData\Local\Temp\ipykernel_14008\1938979547.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  cruce_1_final.drop_duplicates(subset='report_id',keep='last', inplace=True)


,type,merchant_id,merchant_token,merchant_name,user_email,report_id,report_date_local,amount_x,bank_name,payment_date_local,...,tipo_dispersion,tipo_pago,amount_y,ultimo_mov,ultimo_mov_date_only,month,status,error_description,esquema_query,status_cubeta
524042,DAILY,236498a6-d88b-492e-b0b7-f3bc6a5dfea2,319244d0-1867-4862-b42c-8ce5710158ee,Papeleria e Internet Allison,adryromero04@gmail.com,TCS89F3L6,2023-03-05,2.60,"Opciones Empresariales del Noreste, S.A. de C....",2023-03-06 06:51:41,...,TRAD,Pago del día,2.60,2023-05-29,2023-05-29,5.00,ERROR_TV,sin_descripcion,ERROR_USUAL,ERROR
828768,DAILY,41dba6de-cafe-46ea-9ebe-4a9fb2a933df,25ab51da-cdc5-4084-9a83-a7be68c2a1f7,la Reyna Michoacana,materialesdlaparra@hotmail.com,4JH35MXPN,2023-03-09,61.33,"Scotiabank Inverlat, S.A.",2023-03-09 10:14:10,...,TRAD,Pago del día,61.33,2023-05-26,2023-05-26,5.00,ERROR_TV,sin_descripcion,ERROR_USUAL,ERROR
922454,DAILY,41dba6de-cafe-46ea-9ebe-4a9fb2a933df,25ab51da-cdc5-4084-9a83-a7be68c2a1f7,la Reyna Michoacana,materialesdlaparra@hotmail.com,2PHWGCGS9,2023-03-10,36.41,"Scotiabank Inverlat, S.A.",2023-03-10 10:16:41,...,TRAD,Pago del día,36.41,2023-05-26,2023-05-26,5.00,ERROR_TV,sin_descripcion,ERROR_USUAL,ERROR
1118344,DAILY,236498a6-d88b-492e-b0b7-f3bc6a5dfea2,319244d0-1867-4862-b42c-8ce5710158ee,Papeleria e Internet Allison,adryromero04@gmail.com,TQLSVZ5MF,2023-03-11,3.00,"Opciones Empresariales del Noreste, S.A. de C....",2023-03-12 07:01:17,...,TRAD,Pago del día,3.00,2023-05-29,2023-05-29,5.00,ERROR_TV,sin_descripcion,ERROR_USUAL,ERROR
1873239,DAILY,236498a6-d88b-492e-b0b7-f3bc6a5dfea2,319244d0-1867-4862-b42c-8ce5710158ee,Papeleria e Internet Allison,adryromero04@gmail.com,3XJBBY2FH,2023-03-19,0.80,"Opciones Empresariales del Noreste, S.A. de C....",2023-03-20 06:51:12,...,TRAD,Pago del día,0.80,2023-05-29,2023-05-29,5.00,ERROR_TV,sin_descripcion,ERROR_USUAL,ERROR


In [38]:
cruce_1_final.shape

(10455, 27)

In [39]:
duplicates_cruce_1_final = cruce_1_final[cruce_1_final.duplicated(['report_id'] , keep=False)]
duplicates_cruce_1_final.head()

,type,merchant_id,merchant_token,merchant_name,user_email,report_id,report_date_local,amount_x,bank_name,payment_date_local,...,tipo_dispersion,tipo_pago,amount_y,ultimo_mov,ultimo_mov_date_only,month,status,error_description,esquema_query,status_cubeta


In [40]:
duplicates_cruce_1_final.shape

(0, 27)

# CRUCE 2 MES ACTUAL

In [41]:
cruce_2 = query_error_current_month.merge(allfiles[['report_id','tipo_dispersion','tipo_pago']],on='report_id',how='left')
cruce_2['tipo_dispersion'].fillna('Not found',inplace=True)
cruce_2.head()

,Unnamed: 0,merchant_id,merchant_name,report_id,report_date_local,amount,status,error_description,ultimo_mov,month,year,status_cubeta,esquema_query,merchant_id_query,ultimo_mov_date_only,day,mes_cierre,tipo_dispersion,tipo_pago
0,117721,962a6269-3b7b-453a-a973-2b746b0b4909,NaN,APPFQTDKZ,2023-04-29,443.67,ERROR,DEV.SPEIEXCEDE EL LNMITE DE AB,2023-05-01,5,2023,NEW,ERROR_USUAL,962a6269-3b7b-453a-a973-2b746b0b4909,2023-05-01,1,Actual,TRAD,Pago del día
1,118372,af5a71ac-8639-420d-b513-0395c0c4f66d,NaN,ATFHQVKWX,2023-05-01,603.69,ERROR,Bank account information not available,2023-05-01,5,2023,NEW,SIN_CLABE,af5a71ac-8639-420d-b513-0395c0c4f66d,2023-05-01,1,Actual,Not found,NaN
2,118373,1493da6d-4ee7-4b86-a9d6-56b24243f66b,NaN,A62D642LC,2023-05-01,129.36,ERROR,Bank account information not available,2023-05-01,5,2023,NEW,SIN_CLABE,1493da6d-4ee7-4b86-a9d6-56b24243f66b,2023-05-01,1,Actual,Not found,NaN
3,118374,e9513730-acbe-4fa3-964a-f09df2b80c6e,NaN,AGBCM9MZ9,2023-05-01,"1,812.03",ERROR,Bank account information not available,2023-05-01,5,2023,NEW,SIN_CLABE,e9513730-acbe-4fa3-964a-f09df2b80c6e,2023-05-01,1,Actual,Not found,NaN
4,118375,6383fe37-ae8d-47ce-bda5-dc06bf1c176f,NaN,AQ7LGNJD3,2023-05-01,"1,302.25",ERROR,Bank account information not available,2023-05-01,5,2023,NEW,SIN_CLABE,6383fe37-ae8d-47ce-bda5-dc06bf1c176f,2023-05-01,1,Actual,Not found,NaN


In [42]:
cruce_2.shape

(17895, 19)

In [43]:
cruce_2_final = cruce_2[(cruce_2['tipo_dispersion'] != 'Not found')]
#cruce_2_final.drop_duplicates(subset='report_id',keep='last', inplace=True) 
cruce_2_final.head()

,Unnamed: 0,merchant_id,merchant_name,report_id,report_date_local,amount,status,error_description,ultimo_mov,month,year,status_cubeta,esquema_query,merchant_id_query,ultimo_mov_date_only,day,mes_cierre,tipo_dispersion,tipo_pago
0,117721,962a6269-3b7b-453a-a973-2b746b0b4909,NaN,APPFQTDKZ,2023-04-29,443.67,ERROR,DEV.SPEIEXCEDE EL LNMITE DE AB,2023-05-01,5,2023,NEW,ERROR_USUAL,962a6269-3b7b-453a-a973-2b746b0b4909,2023-05-01,1,Actual,TRAD,Pago del día
7,117033,a4609303-9f5d-45af-b6af-82311a68ea7b,NaN,AFQLGX4DB,2023-04-27,268.31,ERROR,DEV.SPEICUENTA BLOQUEADA,2023-05-01,5,2023,NEW,ERROR_USUAL,a4609303-9f5d-45af-b6af-82311a68ea7b,2023-05-01,1,Actual,TRAD,Pago del día
8,117033,a4609303-9f5d-45af-b6af-82311a68ea7b,NaN,AFQLGX4DB,2023-04-27,268.31,ERROR,DEV.SPEICUENTA BLOQUEADA,2023-05-01,5,2023,NEW,ERROR_USUAL,a4609303-9f5d-45af-b6af-82311a68ea7b,2023-05-01,1,Actual,TRAD,Reprogramación
18,117058,1aaf2580-b90b-4784-8d75-6d2c07118c85,NaN,A9429GWCD,2023-04-27,2.00,ERROR,DEV.SPEICUENTA BLOQUEADA,2023-05-01,5,2023,NEW,ERROR_USUAL,1aaf2580-b90b-4784-8d75-6d2c07118c85,2023-05-01,1,Actual,TRAD,Pago del día
19,117058,1aaf2580-b90b-4784-8d75-6d2c07118c85,NaN,A9429GWCD,2023-04-27,2.00,ERROR,DEV.SPEICUENTA BLOQUEADA,2023-05-01,5,2023,NEW,ERROR_USUAL,1aaf2580-b90b-4784-8d75-6d2c07118c85,2023-05-01,1,Actual,TRAD,Reprogramación


In [44]:
cruce_2_final.shape

(12723, 19)

In [45]:
duplicates_cruce_2_final = cruce_2_final[cruce_2_final.duplicated(['report_id'] , keep=False)]
duplicates_cruce_2_final.head()

,Unnamed: 0,merchant_id,merchant_name,report_id,report_date_local,amount,status,error_description,ultimo_mov,month,year,status_cubeta,esquema_query,merchant_id_query,ultimo_mov_date_only,day,mes_cierre,tipo_dispersion,tipo_pago
7,117033,a4609303-9f5d-45af-b6af-82311a68ea7b,NaN,AFQLGX4DB,2023-04-27,268.31,ERROR,DEV.SPEICUENTA BLOQUEADA,2023-05-01,5,2023,NEW,ERROR_USUAL,a4609303-9f5d-45af-b6af-82311a68ea7b,2023-05-01,1,Actual,TRAD,Pago del día
8,117033,a4609303-9f5d-45af-b6af-82311a68ea7b,NaN,AFQLGX4DB,2023-04-27,268.31,ERROR,DEV.SPEICUENTA BLOQUEADA,2023-05-01,5,2023,NEW,ERROR_USUAL,a4609303-9f5d-45af-b6af-82311a68ea7b,2023-05-01,1,Actual,TRAD,Reprogramación
18,117058,1aaf2580-b90b-4784-8d75-6d2c07118c85,NaN,A9429GWCD,2023-04-27,2.00,ERROR,DEV.SPEICUENTA BLOQUEADA,2023-05-01,5,2023,NEW,ERROR_USUAL,1aaf2580-b90b-4784-8d75-6d2c07118c85,2023-05-01,1,Actual,TRAD,Pago del día
19,117058,1aaf2580-b90b-4784-8d75-6d2c07118c85,NaN,A9429GWCD,2023-04-27,2.00,ERROR,DEV.SPEICUENTA BLOQUEADA,2023-05-01,5,2023,NEW,ERROR_USUAL,1aaf2580-b90b-4784-8d75-6d2c07118c85,2023-05-01,1,Actual,TRAD,Reprogramación
27,117080,6535dd34-52f6-4c31-b60c-a022333716a7,NaN,AHGZSXPY9,2023-04-27,52.70,ERROR,DEV.SPEICUENTA BLOQUEADA,2023-05-01,5,2023,NEW,ERROR_USUAL,6535dd34-52f6-4c31-b60c-a022333716a7,2023-05-01,1,Actual,TRAD,Pago del día


In [46]:
duplicates_cruce_2_final.shape

(3916, 19)

# AGRUPADO POR TIPO DE PAGO - MES ACTUAL

In [47]:
group_tipo_pago = pd.pivot_table(cruce_1_final,index=['ultimo_mov_date_only']
                                              ,columns=['esquema_query','status_cubeta','tipo_dispersion',
                                                        'tipo_pago'],values=['amount_x']
                                              ,aggfunc=np.sum,margins=True).reset_index()
group_tipo_pago.fillna(0,inplace=True)
group_tipo_pago.head()

ultimo_mov_date_only       amount_x               \
esquema_query                           ERROR_USUAL                
status_cubeta                                 ERROR                
tipo_dispersion                                  FP         TRAD   
tipo_pago                            Reprogramación Pago del día   
0                         2023-05-01           0.00         0.00   
1                         2023-05-02      10,826.77         0.00   
2                         2023-05-03           0.00         0.00   
3                         2023-05-04           0.00         0.00   
4                         2023-05-05           0.00         0.00   

                                                                         \
esquema_query                                                             
status_cubeta                           NEW                               
tipo_dispersion                          FP                        TRAD   
tipo_pago       Reprogramación Pago del día Reprogramación Pago del día   
0                         0.00    99,114.84           0.00   146,858.82   
1                         0.00    82,420.53         520.42   170,339.30   
2                     1,224.40       648.11           0.00    64,547.11   
3                         0.00    10,065.57       1,745.89    50,320.64   
4                         0.00     1,498.67       7,666.14    54,571.24   

                                           
esquema_query                         All  
status_cubeta                              
tipo_dispersion                            
tipo_pago       Reprogramación             
0                       323.01 246,296.67  
1                         0.00 264,107.02  
2                         0.00  66,419.62  
3                         0.00  62,132.10  
4                         0.00  63,736.05

In [48]:
group_tipo_pago.shape

(32, 9)

In [49]:
group_esquema = pd.pivot_table(cruce_1_final,index=['ultimo_mov_date_only']
                                              ,columns=['esquema_query','status_cubeta','tipo_pago'],values=['amount_x']
                                              ,aggfunc=np.sum,margins=True).reset_index()
group_esquema.fillna(0,inplace=True)
group_esquema.head()

ultimo_mov_date_only     amount_x                              \
esquema_query                       ERROR_USUAL                               
status_cubeta                             ERROR                         NEW   
tipo_pago                          Pago del día Reprogramación Pago del día   
0                       2023-05-01         0.00           0.00   245,973.66   
1                       2023-05-02         0.00      10,826.77   252,759.83   
2                       2023-05-03         0.00       1,224.40    65,195.22   
3                       2023-05-04         0.00           0.00    60,386.21   
4                       2023-05-05         0.00           0.00    56,069.91   

                                         
esquema_query                       All  
status_cubeta                            
tipo_pago     Reprogramación             
0                     323.01 246,296.67  
1                     520.42 264,107.02  
2                       0.00  66,419.62  
3                   1,745.89  62,132.10  
4                   7,666.14  63,736.05

In [50]:
group_esquema.shape

(32, 6)

# CRUCE 3 - MES ANTERIOR

In [51]:
cruce_3 = allfiles.merge(query_new_last_month[['report_id','amount','ultimo_mov','ultimo_mov_date_only','month','status',
                                               'error_description','esquema_query','status_cubeta']],
                         on='report_id',how='left')
cruce_3['month'].fillna('Not found',inplace=True)
cruce_3.head()

,type,merchant_id,merchant_token,merchant_name,user_email,report_id,report_date_local,amount_x,bank_name,payment_date_local,...,tipo_dispersion,tipo_pago,amount_y,ultimo_mov,ultimo_mov_date_only,month,status,error_description,esquema_query,status_cubeta
0,DAILY,652400c6-31ac-479d-a667-0db7225b1c0a,80af32bb-cd90-4d11-a3da-126ea9822fcb,Abarrotes Flor,iro_68@hotmail.com,22237PFT4,2023-03-01,279.51,BANAMEX,2023-03-01 11:49:47,...,TRAD,Pago del día,NaN,NaT,NaN,Not found,NaN,NaN,NaN,NaN
1,DAILY,82ca1460-c459-41a1-ba50-f412dfd2d12a,e899a771-2913-4e6e-bd1e-89598302a789,TheBarbersLM,lorenamendez21461@hotmail.com,222D4MHCS,2023-03-01,383.30,BBVA Bancomer,2023-03-01 11:46:47,...,TRAD,Pago del día,NaN,NaT,NaN,Not found,NaN,NaN,NaN,NaN
2,DAILY,3e3d2b7d-eed6-4dd5-b251-0aa310f0c85f,a6a3d955-a08c-4b6d-bccb-71d498ed2528,Dulces andy,jrgk7232@gmail.com,222KVTC6P,2023-03-01,"1,437.36",Banco Azteca,2023-03-01 11:43:46,...,TRAD,Pago del día,NaN,NaT,NaN,Not found,NaN,NaN,NaN,NaN
3,DAILY,031c670b-6626-40a1-89ea-fc5ed1bbec8d,99d051b9-bceb-46a4-a82f-21cac8c9530b,Eridental,draerival@hotmail.com,222L2BHB2,2023-03-01,"2,491.42",SANTANDER,2023-03-01 11:42:10,...,TRAD,Pago del día,NaN,NaT,NaN,Not found,NaN,NaN,NaN,NaN
4,DAILY,6bce3214-9af4-4c92-a5e0-dfc7f749e55e,2894580e-15a6-4fbe-ae0b-3cf25b402ec1,Abarrotes salas,m.josegabriel38@gmail.com,222N6GY5F,2023-03-01,35.46,BBVA Bancomer,2023-03-01 11:42:58,...,TRAD,Pago del día,NaN,NaT,NaN,Not found,NaN,NaN,NaN,NaN


In [52]:
cruce_3.shape

(9352472, 27)

In [53]:
cruce_3_final = cruce_3[(cruce_3['month'] != 'Not found')]
cruce_3_final.drop_duplicates(subset='report_id',keep='last', inplace=True) 
cruce_3_final.head()

,type,merchant_id,merchant_token,merchant_name,user_email,report_id,report_date_local,amount_x,bank_name,payment_date_local,...,tipo_dispersion,tipo_pago,amount_y,ultimo_mov,ultimo_mov_date_only,month,status,error_description,esquema_query,status_cubeta


In [54]:
cruce_3_final.shape

(0, 27)

In [55]:
duplicates_cruce_3_final = cruce_3_final[cruce_3_final.duplicated(['report_id'] , keep=False)]
duplicates_cruce_3_final.head()

,type,merchant_id,merchant_token,merchant_name,user_email,report_id,report_date_local,amount_x,bank_name,payment_date_local,...,tipo_dispersion,tipo_pago,amount_y,ultimo_mov,ultimo_mov_date_only,month,status,error_description,esquema_query,status_cubeta


In [56]:
duplicates_cruce_3_final.shape

(0, 27)

# CRUCE 4 - MES ANTERIOR

In [57]:
cruce_4 = query_new_last_month.merge(allfiles[['report_id','tipo_dispersion','tipo_pago']],on='report_id',how='left')
cruce_4['tipo_dispersion'].fillna('Not found',inplace=True)
cruce_4.head()

,Unnamed: 0,merchant_id,merchant_name,report_date_local,amount,status,error_description,ultimo_mov,month,year,status_cubeta,esquema_query,merchant_id_query,ultimo_mov_date_only,day,mes_cierre,report_id,tipo_dispersion,tipo_pago


In [58]:
cruce_4.shape

(0, 19)

In [59]:
cruce_4_final = cruce_4[(cruce_4['tipo_dispersion'] != 'Not found')]
#cruce_2_final.drop_duplicates(subset='report_id',keep='last', inplace=True) 
cruce_4_final.head()

,Unnamed: 0,merchant_id,merchant_name,report_date_local,amount,status,error_description,ultimo_mov,month,year,status_cubeta,esquema_query,merchant_id_query,ultimo_mov_date_only,day,mes_cierre,report_id,tipo_dispersion,tipo_pago


In [60]:
cruce_4_final.shape

(0, 19)

In [61]:
duplicates_cruce_4_final = cruce_4_final[cruce_4_final.duplicated(['report_id'] , keep=False)]
duplicates_cruce_4_final.head()

,Unnamed: 0,merchant_id,merchant_name,report_date_local,amount,status,error_description,ultimo_mov,month,year,status_cubeta,esquema_query,merchant_id_query,ultimo_mov_date_only,day,mes_cierre,report_id,tipo_dispersion,tipo_pago


In [62]:
duplicates_cruce_4_final.shape

(0, 19)

# AGRUPADO POR TIPO DE PAGO - MES ANTERIOR

In [63]:
group_tipo_pago_last_month = pd.pivot_table(cruce_3_final,index=['ultimo_mov_date_only']
                                              ,columns=['esquema_query','status_cubeta','tipo_dispersion',
                                                        'tipo_pago'],values=['amount_x']
                                              ,aggfunc=np.sum,margins=True).reset_index()
group_tipo_pago_last_month.fillna(0,inplace=True)
group_tipo_pago_last_month.head()

ValueError: No objects to concatenate

In [64]:
group_tipo_pago_last_month.shape

NameError: name 'group_tipo_pago_last_month' is not defined

In [65]:
group_esquema_last_month = pd.pivot_table(cruce_4_final,index=['ultimo_mov_date_only']
                                              ,columns=['esquema_query','status_cubeta','tipo_pago'],values=['amount']
                                              ,aggfunc=np.sum,margins=True).reset_index()
group_esquema_last_month.fillna(0,inplace=True)
group_esquema_last_month.head()

ValueError: No objects to concatenate

In [66]:
group_esquema_last_month.shape

NameError: name 'group_esquema_last_month' is not defined

In [69]:
writer = pd.ExcelWriter('FASE V. DATA POLIZA ERROR-USUAL MAYO 2023.xlsx',
                        engine='xlsxwriter',
                        engine_kwargs={'options': {'encoding':'utf-8'}})
group_tipo_pago.to_excel(writer, sheet_name='GRUPO TIPO PAGO')
group_esquema.to_excel(writer, sheet_name='GRUPO ESQUEMA')
group_tipo_pago_last_month.to_excel(writer, sheet_name='GRUPO TIPO PAGO-MESES ANT')
group_esquema_last_month.to_excel(writer, sheet_name='GRUPO ESQUEMA-MESES ANT')
cruce_1_final.to_excel(writer, sheet_name='DETALLE',index=False)
cruce_3_final.to_excel(writer, sheet_name='DETALLE-MESES ANTERIORES',index=False)
writer.close()